Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "Bill Zhang"
COLLABORATORS = ""

---

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "Bill Zhang"
COLLABORATORS = ""

---

In [76]:
pip install mysql-connector-python


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 34.7 MB/s eta 0:00:0000:0100:01m
Note: you may need to restart the kernel to use updated packages.


In [39]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, timezone
import sqlite3
import requests

# Simulate synthetic weather data with latitude and longitude
def generate_synthetic_weather_data(start_date, latitude, longitude, days=5):
    data = []
    current_date = start_date
    for _ in range(days * 24):  # 24 hours per day
        # Simulate temperature with seasonal and latitude effects
        base_temp = 30 - abs(latitude) * 0.3  # Lower temperatures at higher latitudes
        temperature = base_temp + 10 * np.sin(2 * np.pi * current_date.timetuple().tm_yday / 365) + np.random.normal(0, 2)
        
        # Simulate humidity inversely correlated with temperature
        humidity = np.clip(100 - temperature + np.random.normal(0, 5), 20, 100)
        
        # Simulate wind speed with random variations
        wind_speed = np.random.uniform(0, 15)
        
        # Append data
        data.append([current_date, temperature, humidity, wind_speed, latitude, longitude])
        current_date += timedelta(hours=1)  # Increment time by 1 hour

    # Convert to DataFrame
    return pd.DataFrame(data, columns=["timestamp", "temperature", "humidity", "wind_speed", "latitude", "longitude"])

# Generate synthetic data for the past 30 days with specific latitude and longitude
latitude, longitude = 40.7128, -74.0060  # Example: New York City
start_date = datetime.now(timezone.utc) - timedelta(days=30)
synthetic_data = generate_synthetic_weather_data(start_date, latitude, longitude, days=30)

# Display a preview of the synthetic data
print("Simulated Historical Data:")
print(synthetic_data.head())

# 1. Initialize SQLite Database and Create Table
def initialize_database():
    conn = sqlite3.connect("weather_data.db")  # Create or connect to the SQLite database
    cursor = conn.cursor()
    
    # Create a table for storing weather data
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS weather (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            timestamp TEXT,
            temperature REAL,
            humidity REAL,
            wind_speed REAL,
            latitude REAL,
            longitude REAL
        )
    ''')
    conn.commit()
    conn.close()

# 2. Insert Synthetic Data into SQLite
def store_synthetic_data(data, connection):
    cursor = connection.cursor()
    # Insert data into the weather table
    for _, row in data.iterrows():
        cursor.execute("""
            INSERT INTO weather (timestamp, temperature, humidity, wind_speed, latitude, longitude)
            VALUES (?, ?, ?, ?, ?, ?)
        """, (
            row['timestamp'].isoformat(),  # Convert timestamp to ISO format
            row['temperature'],
            row['humidity'],
            row['wind_speed'],
            row['latitude'],
            row['longitude']
        ))
    connection.commit()
    cursor.close()

    print("Synthetic data stored in SQLite database.")

# 3. Fetch Current Weather Data
def fetch_current_weather(lat, lon, api_key):
    CURRENT_WEATHER_API_URL = "https://api.openweathermap.org/data/2.5/weather"
    try:
        params = {'lat': lat, 'lon': lon, 'appid': api_key, 'units': 'metric'}
        response = requests.get(CURRENT_WEATHER_API_URL, params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching current weather data: {e}")
        return None

# 4. Insert Current Weather Data into SQLite
def store_current_weather(data, latitude, longitude):
    conn = sqlite3.connect("weather_data.db")
    cursor = conn.cursor()
    
    # Insert current weather data into the weather table
    cursor.execute('''
        INSERT INTO weather (timestamp, temperature, humidity, wind_speed, latitude, longitude)
        VALUES (?, ?, ?, ?, ?, ?)
    ''', (
        datetime.now().isoformat(),
        data['main']['temp'],
        data['main']['humidity'],
        data['wind']['speed'],
        latitude,
        longitude
    ))
    
    conn.commit()
    conn.close()
    print("Current weather data stored in SQLite database.")

# Initialize database
initialize_database()
conn = sqlite3.connect("weather_data.db") 

# Store synthetic data
store_synthetic_data(synthetic_data, conn)

# Fetch and store current weather data
API_KEY = "d3f2be05b1b69cd6e54b7834dd76279b"
latitude, longitude = 40.7128, -74.0060
current_weather = fetch_current_weather(latitude, longitude, API_KEY)
if current_weather:
    store_current_weather(current_weather, latitude, longitude)

print("Data storage complete.")


Simulated Historical Data:
                         timestamp  temperature   humidity  wind_speed  \
0 2024-11-09 22:56:54.601174+00:00     5.967224  98.646639    0.811841   
1 2024-11-09 23:56:54.601174+00:00    10.861152  87.351854    0.718534   
2 2024-11-10 00:56:54.601174+00:00    11.603989  87.573729    6.995692   
3 2024-11-10 01:56:54.601174+00:00    10.804563  83.373149    2.247883   
4 2024-11-10 02:56:54.601174+00:00    10.560969  93.445178   14.800977   

   latitude  longitude  
0   40.7128    -74.006  
1   40.7128    -74.006  
2   40.7128    -74.006  
3   40.7128    -74.006  
4   40.7128    -74.006  
Synthetic data stored in SQLite database.
Current weather data stored in SQLite database.
Data storage complete.


In [40]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from datetime import datetime, timedelta
import sqlite3
import numpy as np
from statsmodels.tsa.arima.model import ARIMA

# Ensure all columns are displayed when printing DataFrames
pd.set_option('display.max_columns', None)

# Query data from SQLite
def query_data_from_sqlite(database_path="weather_data.db"):
    conn = sqlite3.connect(database_path)
    query = "SELECT timestamp, temperature, humidity, wind_speed, latitude, longitude FROM weather"
    data = pd.read_sql_query(query, conn)
    conn.close()
    return data

# Fetch historical weather data
historical_data_df = query_data_from_sqlite()

# Validate required columns
required_columns = ['timestamp', 'latitude', 'longitude', 'temperature', 'humidity', 'wind_speed']
missing_columns = [col for col in required_columns if col not in historical_data_df.columns]
if missing_columns:
    raise ValueError(f"Missing required columns: {missing_columns}")

# Normalize timestamps
historical_data_df['timestamp'] = pd.to_datetime(
    historical_data_df['timestamp'], errors='coerce'
)

# Drop rows with invalid timestamps
historical_data_df = historical_data_df.dropna(subset=['timestamp'])

# Set timestamp as the index and ensure it is properly sorted
historical_data_df = historical_data_df.set_index('timestamp').sort_index()

# Assign a frequency to the data for ARIMA (assuming hourly data)
historical_data_df = historical_data_df.asfreq('h')  # Use lowercase 'h' for hourly frequency

# Drop rows with missing values in required columns
historical_data_df = historical_data_df.dropna(subset=['temperature', 'humidity', 'wind_speed', 'latitude', 'longitude'])

# Prepare data for ARIMA
temperature_series = historical_data_df['temperature']

# Train ARIMA model
arima_model = ARIMA(temperature_series, order=(2, 1, 2))  # Adjust order based on your dataset
arima_model_fit = arima_model.fit()

# Predict for the next 7 days (hourly predictions)
future_steps = 7 * 24  # 7 days × 24 hours
arima_forecast = arima_model_fit.forecast(steps=future_steps)

# Prepare hourly timestamps for 7 days
future_dates = pd.date_range(start=historical_data_df.index[-1] + timedelta(hours=1), periods=future_steps, freq='h')

# Create forecast DataFrame
forecast_df = pd.DataFrame({
    'timestamp': future_dates,
    'predicted_temperature': arima_forecast
})

# Print forecast
print("Forecasted Temperatures for the Next 7 Days:")
print(forecast_df)

# Add dummy adjustments for other variables (humidity, wind speed)
forecast_df['predicted_humidity'] = np.clip(100 - forecast_df['predicted_temperature'] + np.random.normal(0, 5, size=future_steps), 20, 100)
forecast_df['predicted_wind_speed'] = np.random.uniform(0, 15, size=future_steps)

# Display the forecast with additional variables
print("\nForecasted Weather Conditions for the Next 7 Days:")
print(forecast_df.head(24))  # Display first 24 hours of predictions


Forecasted Temperatures for the Next 7 Days:
                                                        timestamp  \
2024-12-08 23:36:04.535993+00:00 2024-12-08 23:36:04.535993+00:00   
2024-12-09 00:36:04.535993+00:00 2024-12-09 00:36:04.535993+00:00   
2024-12-09 01:36:04.535993+00:00 2024-12-09 01:36:04.535993+00:00   
2024-12-09 02:36:04.535993+00:00 2024-12-09 02:36:04.535993+00:00   
2024-12-09 03:36:04.535993+00:00 2024-12-09 03:36:04.535993+00:00   
...                                                           ...   
2024-12-15 18:36:04.535993+00:00 2024-12-15 18:36:04.535993+00:00   
2024-12-15 19:36:04.535993+00:00 2024-12-15 19:36:04.535993+00:00   
2024-12-15 20:36:04.535993+00:00 2024-12-15 20:36:04.535993+00:00   
2024-12-15 21:36:04.535993+00:00 2024-12-15 21:36:04.535993+00:00   
2024-12-15 22:36:04.535993+00:00 2024-12-15 22:36:04.535993+00:00   

                                  predicted_temperature  
2024-12-08 23:36:04.535993+00:00              13.619941  
2024-12-09